In [1]:
%load_ext autoreload
%autoreload 2
import telepot
bot = telepot.Bot('436447719:AAGq_iK2hE3cPFhmL9Jh53g9HyVkw3SXZbg')
def qq(txt):
    bot.sendMessage(1114926, txt)

In [99]:
qq('start')

In [100]:
import json
import gzip
from itertools import islice
from collections import Counter, defaultdict
from operator import itemgetter
import pyarrow.parquet as pq
import pyarrow as pa

import pandas as pd
from tqdm.notebook import tqdm
import dill
import os
from pqdm.processes import pqdm
import re

from workers import *
    
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec

import random
import collections
import gc

In [108]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score

In [5]:
GLOBAL_DEVICE =  torch.device('cpu') # torch.device('cuda:0') # 

In [47]:
class My_Iterator(object):
    def __init__(self, df, batch=64, vect_name='text_vector', label_name='specialization'):
        cols_to_drop = [col for col in df.columns if col not in (vect_name, label_name)]
        self.df = df.reset_index(drop=True).copy()
        self.df.drop(columns=cols_to_drop, inplace=True)
        self.batch = batch 
        self.len = df.shape[0]
        self.vect_name = vect_name
        self.label_name = label_name

    def __iter__(self):
        j = -1
        for i in range(0, self.len, self.batch):
            vec = torch.tensor(self.df.loc[i:(i+self.batch-1), self.vect_name].to_list())
            lbl = torch.tensor(self.df.loc[i:(i+self.batch-1), self.label_name].values)
            j += 1
            yield j, vec, lbl

In [13]:
def evaluate_model(model, iterator, number_of_batches = 10):
    all_preds = []
    all_y = []
    model.train(False)
    for i, x, y in iterator:
        x = x.to(GLOBAL_DEVICE)
        y_pred = model(x)
        predicted = torch.max(y_pred.cpu().data, 1)[1]
        all_preds.extend(predicted.numpy())
        all_y.extend(y.numpy())
    model.train(True)
    score = accuracy_score(all_y, np.array(all_preds).flatten())
    return score

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, config):
        super(SimpleNN, self).__init__()
        self.config = config
        self.device= GLOBAL_DEVICE
        
        # Input:  batch_size x input_size 
        # Output: batch_size x hidden_size
        
#         # Fully-Connected Layer
#         self.fc0 = nn.Linear(
#             self.config.input_size,
#             self.config.output_size[0]
#         )
        
#         self.fc1 = nn.Linear(
#             self.config.output_size[0],
#             self.config.output_size[1]
#         )
        
#         self.fc2 = nn.Linear(
#             self.config.output_size[1],
#             self.config.output_size[2]
#         )
        
        # Softmax non-linearity
        self.logsoftmax = nn.LogSoftmax(dim=1)
        
        self.classifier = nn.Sequential(
            nn.Linear(self.config.input_size, self.config.output_size[0]),
            nn.Dropout(0.2)
            nn.BatchNorm1d(self.config.output_size[0]), #applying batch norm
            nn.ReLU(),
            nn.Linear(self.config.output_size[0], self.config.output_size[1]),
            nn.Dropout(0.2)
            nn.BatchNorm1d(self.config.output_size[1]),
            nn.ReLU(),
            nn.Linear(self.config.output_size[1], self.config.output_size[2])
            nn.LogSoftmax(dim=1)
        )
    
        
    def forward(self, x):
        # x.shape = (input_size, batch_size)
        
#         f0_out = self.fc0(x)
        
#         f0_out = F.relu(f0_out)

#         f1_out = self.fc1(f0_out)
        
#         f1_out = F.relu(f1_out)

#         f2_out = self.fc2(f1_out)

#         return self.logsoftmax(f2_out)
        return self.classifier(x)
    
    def add_optimizer(self, optimizer):
        self.optimizer = optimizer
        
    def add_loss_op(self, loss_op):
        self.loss_op = loss_op
    
    def reduce_lr(self):
        print("Reducing LR")
        for g in self.optimizer.param_groups:
            g['lr'] = g['lr'] / 2
        print("Reducing LR:", [g['lr'] for gr in self.optimizer.param_groups])
                
    def run_epoch(self, train_iterator, test_iterator, epoch, bt_count):
        train_losses = []
        test_accuracies = []
        losses = []
        total = 0
        
        accuracies = []
        
        # Reduce learning rate as number of epochs increase
        if (epoch == int(self.config.max_epochs/3)) or (epoch == int(2*self.config.max_epochs/3)):
            self.reduce_lr()
            
        for i, x, y in train_iterator:
          
            x = Variable(x.to(self.device))
            y = y.to(self.device)
            bt_count += 1
            
            self.optimizer.zero_grad()
            y_pred = self.__call__(x)
                    
            loss = self.loss_op(y_pred, y)
            loss.backward()
            losses.append(loss.data.cpu().numpy())
            #accuracy = accuracy_score(y.data.cpu().numpy(), torch.max(y_pred.cpu().data, 1)[1])
            self.optimizer.step()
    
            if i % 100 == 0:
                print("Iter: {}".format(i+1))
                avg_train_loss = np.mean(losses)
                train_losses.append(avg_train_loss)
                print("\tAverage training loss: {:.5f}".format(avg_train_loss))
                losses = []
                
                # Evalute Accuracy on test set
                test_accuracy = evaluate_model(self, test_iterator)
                print("\tTest Accuracy: {:.4f}".format(test_accuracy))
                writer.add_scalar('Loss/Train', avg_train_loss, bt_count)
                writer.add_scalar('Accuracy/Test', test_accuracy, bt_count)
                
                self.train()
                             
        return train_losses, test_accuracy, bt_count

In [7]:
class Config(object):
    output_size = [300, 460 , 620]
    max_epochs = 5
    lr = 0.5
    batch_size = 64
    input_size = 300


config = Config()

In [10]:
model=None
gc.collect()

40

In [24]:
# load df
df = pd.read_parquet('df_unp_vector.pq')

In [28]:
test_vac_id = set(dill.load(open('validatin.bin','rb')))

In [29]:
df['validation'] = df['vacancy_id'].apply(lambda x: x in test_vac_id)

In [30]:
df.columns

Index(['vacancy_id', 'employer', 'key_list', 'name_list', 'text_vector',
       'specialization', 'validation'],
      dtype='object')

In [31]:
df.head()

,vacancy_id,employer,key_list,name_list,text_vector,specialization,validation
0,2700005,845fe6266ab1,[],"[Разработчик, Python/django]","[-0.37388623, 0.26167643, 0.3595109, -0.166157...",137,False
1,2700005,845fe6266ab1,[],"[Разработчик, Python/django]","[-0.37388623, 0.26167643, 0.3595109, -0.166157...",221,False
2,2700005,845fe6266ab1,[],"[Разработчик, Python/django]","[-0.37388623, 0.26167643, 0.3595109, -0.166157...",296,False
3,2700006,23ead4a8f164,"[работа, текущий, база, клиент, мерчендайзинг,...","[Торговый, представитель]","[-0.07712369, -0.11375963, 0.07628894, -0.0080...",149,False
4,2700006,23ead4a8f164,"[работа, текущий, база, клиент, мерчендайзинг,...","[Торговый, представитель]","[-0.07712369, -0.11375963, 0.07628894, -0.0080...",242,False


In [88]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [90]:
df['specialization'] = le.fit_transform(df['specialization'])

In [96]:
dill.dump(le, open('le.bin', 'wb'))

In [109]:
%load_ext tensorboard

In [123]:
!rm -rf ./runs/
writer = SummaryWriter('runs/my_run')
if GLOBAL_DEVICE != torch.device('cpu'):
    print("Run on CUDA")
    model.cuda()
else:
    print("Run on CPU")

train_losses = []
test_accuracies = []

Run on CPU


In [124]:

model = SimpleNN(config)

model.train(True)
optimizer = optim.SGD(model.parameters(), lr=config.lr)
NLLLoss = nn.NLLLoss()
model.add_optimizer(optimizer)
model.add_loss_op(NLLLoss)
num_steps = 0

In [ ]:
for i in range(1, config.max_epochs):
    test_iter = My_Iterator(df[df.validation], config.batch_size)
    train_iter = My_Iterator(df[~df.validation], config.batch_size)
    print ("Epoch: {}".format(i))
    train_loss,test_accuracy, num_steps = model.run_epoch(train_iter, test_iter, i, num_steps)
    train_losses.append(train_loss)
    test_accuracies.append(test_accuracy)

train_acc = evaluate_model(model, train_iter)
test_acc = evaluate_model(model, test_iter)

print ('Final Training Accuracy: {:.4f}'.format(train_acc))
print ('Final Test Accuracy: {:.4f}'.format(test_acc))

Epoch: 1
Reducing LR
Reducing LR: [0.25]
Iter: 1
	Average training loss: 6.43412
	Test Accuracy: 0.0021
Iter: 101
	Average training loss: 5.98224
	Test Accuracy: 0.0158
Iter: 201
	Average training loss: 5.64956
	Test Accuracy: 0.0360
Iter: 301
	Average training loss: 5.53619
	Test Accuracy: 0.0480
Iter: 401
	Average training loss: 5.44885
	Test Accuracy: 0.0533
Iter: 501
	Average training loss: 5.24884
	Test Accuracy: 0.0555
Iter: 601
	Average training loss: 5.13826
	Test Accuracy: 0.0641
Iter: 701
	Average training loss: 5.02860
	Test Accuracy: 0.0690
Iter: 801
	Average training loss: 4.98660
	Test Accuracy: 0.0761
Iter: 901
	Average training loss: 4.84567
	Test Accuracy: 0.0804
Iter: 1001
	Average training loss: 4.73747
	Test Accuracy: 0.0725
Iter: 1101
	Average training loss: 4.70895
	Test Accuracy: 0.0810
Iter: 1201
	Average training loss: 4.63732
	Test Accuracy: 0.0836
Iter: 1301
	Average training loss: 4.54997
	Test Accuracy: 0.0865
Iter: 1401
	Average training loss: 4.50931
	Tes

In [73]:
data = torch.randn(5)

In [127]:
torch.save(model, 'fc_model.bin')

/home/rsgbank/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type SimpleNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/rsgbank/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/rsgbank/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LogSoftmax. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/rsgbank/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type NLLLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ 

In [128]:
md = torch.load('fc_model.bin')

In [129]:
md

SimpleNN(
  (fc0): Linear(in_features=300, out_features=300, bias=True)
  (fc1): Linear(in_features=300, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=620, bias=True)
  (logsoftmax): LogSoftmax()
  (loss_op): NLLLoss()
)

In [132]:
md.fc1.weight.shape

torch.Size([500, 300])